In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/date=2010-07-04/device_code=ios-all/
 
 


In [0]:

import datetime
start = "2010-07-04"
end = "2010-08-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

for day in dates:
    next_day = day+datetime.timedelta(1)
    pre_day = day+datetime.timedelta(-1)

    # unified_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(pre_day)).cache()
    unified_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(day)).cache()
    unified_2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(next_day)).cache()




    cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}".format(day)).cache()
    cdf2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}".format(next_day)).cache()

    cdf1.createOrReplaceTempView("cur")
    cdf2.createOrReplaceTempView("next")
    unified_1.createOrReplaceTempView("unified_cur")
    unified_2.createOrReplaceTempView("unified_next")

    print "compare app id: " , day
    spark.sql("select app_id from cur except all select app_id from  next ").show()
    spark.sql("select app_id from (select app_id from cur except all select app_id from next ) as cum1 except all select app_id from ( select app_id from unified_cur except all select app_id from unified_next ) as cum2   ").show()


    # print "compare sum data: ", day
    # spark.sql("select * from unified_cur ").show()
    # spark.sql("select app_id, country_code, sum(free_app_download) as total_free_app_download , sum(revenue) as total_revenue, sum(paid_app_download) as total_paid_app_download, device_code  from (  select country_code,app_id,revenue,free_app_download, paid_app_download, device_code from unified_cur  union select country_code,app_id,revenue, free_app_download, paid_app_download ,device_code from unified_next ) as union_data group by country_code,device_code,app_id   ").show()




# cdf2 = 

In [0]:

import datetime

start = "2010-07-04"
end = "2010-08-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

print dates

unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % (",".join([d.strftime("%Y-%m-%d") for d in dates]))).select("app_id", "country_code", "free_app_download" ,"paid_app_download","revenue", "device_code")

cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}/".format(dates[-1])).cache()

unified_data.createOrReplaceTempView("unified_data")
cdf1.createOrReplaceTempView("cumu")

print "compare sum data: ", dates[-1]

spark.sql("select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code  from unified_data  group by country_code,device_code,app_id except all select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code from cumu  ").show()

spark.sql("select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code from cumu except all select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code  from unified_data  group by country_code,device_code,app_id ").show()


In [0]:

import datetime

start = "2010-07-04"
end = "2010-07-14"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={%s}" % (",".join([d.strftime("%Y-%m-%d") for d in dates]))).select("app_id", "country_code", "free_app_download" ,"paid_app_download","revenue", "device_code","category_id")
test_est_category.createOrReplaceTempView("category_unified_data")
spark.sql("select * from category_unified_data where app_id=283519081 and category_id=100000 and country_code='AE' and device_code='ios-all'").show()




 
test_est_category = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-cum.v1/fact/date=2010-07-10").cache()
test_est_category.createOrReplaceTempView("category")
spark.sql("select * from category where app_id=283519081 and category_id=100000 and country_code='AE' and device_code='ios-all'").show()

In [0]:



spark.sql(" select app_id, country_code, sum(free_app_download) as total_free_app_download , sum(paid_app_download) as toal_paid_app_download, sum(revenue) as total_revenue, device_code  from (  select country_code,app_id,revenue,free_app_download, paid_app_download, device_code from unified_pre_two  union all select country_code,app_id,revenue, free_app_download, paid_app_download ,device_code from unified_pre  union all select country_code,app_id,revenue, free_app_download, paid_app_download ,device_code from unified_cur ) as union_data group by country_code,device_code,app_id").where("app_id='1441499274' and country_code='MT' and device_code='ios-phone'").show()


In [0]:

start = "2019-07-01"
end = "2019-07-05"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

app_id=["1441499274","960832727","283519081"]
unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % (",".join([d.strftime("%Y-%m-%d") for d in dates]))).select("app_id", "country_code", "free_app_download" ,"paid_app_download","revenue", "device_code").filter("app_id in ( %s )" % ",".join(app_id)).cache()

unified_data.createOrReplaceTempView("unified")
spark.sql("select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code from unified group by country_code,device_code,app_id ").show()


In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
import datetime

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()



start = "2010-07-04"
end = "2010-08-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

est_list = [ "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates ]
rank_list = ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates  ]
# print est_list
# print rank_list






def test_count(test_date):
    # test_date = '2017-08-01'
    namespace = "aa.store.market-size.v1"
    
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                # "path": [
                #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                #         test_date)],
                "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                # "path": [
                #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                #         test_date)],
                "path": rank_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "category_mapping_deminsion_service",
                "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "ios_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "android_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
            }
        ]
    }
    
    sql_text = """
    
    
    -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download  , 
                                    store_unified.paid_app_download , 
                                    store_unified.revenue           , 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download AS rank_free_app_download, 
                                    rank_unified.paid_app_download AS rank_paid_app_download, 
                                    rank_unified.revenue AS rank_revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    """
    


    run(spark, ingest_msg, sql_text)
 
    category_1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-cum.v1/fact").where("date='{}'".format(test_date)).cache()


    category_1.createOrReplaceTempView("cumu_category")
    spark.sql("select * from cumu_category limit 5").show()

    spark.sql("select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code, category_id  from unified_data_test  group by country_code,device_code,app_id,category_id  except all select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code,category_id from cumu_category   ").show()
    
    # spark.sql("select count(*) from (select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code, category_id  from unified_data_test  group by country_code,device_code,app_id,category_id  except all select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code,category_id from cumu_category) as prod   ").show()

    spark.sql("select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code,category_id from cumu_category except all select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code, category_id  from unified_data_test  group by country_code,device_code,app_id,category_id ").show()

    # spark.sql("select count(*) from ( select app_id, country_code, free_app_download ,paid_app_download,revenue, device_code,category_id from cumu_category except all select app_id, country_code, sum(free_app_download) as free_app_download , sum(paid_app_download) as paid_app_download  , sum(revenue) as revenue, device_code, category_id  from unified_data_test  group by country_code,device_code,app_id,category_id ) as new ").show()

test_count(dates[-1])


In [0]:

import datetime
from pyspark.sql.types import StructType, StructField, StringType
start = "2020-01-01"
end = "2020-04-19"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


test_ios_app_list = [377194688 , 375875657, 366236510, 354990881,365691077 , 343200656 ,20600000009072,20600000000553]
device_code=['ios-phone','ios-tablet','ios-all','android-all']
country_list=['WW', 'US']
# test_list = [(app, device, country ) for app in test_ios_app_list  for device in device_code for country in country_list ]


sql_where = "app_id in ({}) and device_code in ('{}') and country_code in ('{}')".format(",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list) )
print sql_where


_unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=20{10,11,12,13,14,15,16,17,18,19}-*").where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue")

result=list()
for day in dates:
    _unified_data_single_date = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % (day.strftime("%Y-%m-%d"))).where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue").collect()
    # print _unified_data
    result.extend(_unified_data_single_date)

schema = StructType([StructField("app_id", StringType(), True), 
                     StructField("device_code", StringType(), True),
                     StructField("country_code", StringType(), True),
                     StructField("free_app_download", IntegerType(), True),
                     StructField("paid_app_download", IntegerType(), True),
                     StructField("revenue", IntegerType(), True)])

_unified_data.createOrReplaceTempView("unified_data_view")
spark.createDataFrame(result, schema=schema).createOrReplaceTempView("unified_data_singe_date_view")
spark.sql("select * from unified_data_view union select * from unified_data_singe_date_view").createOrReplaceTempView("union_unified_data")
spark.sql("select app_id, sum(free_app_download) from union_unified_data group by app_id, device_code, country_code").show(2)



cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}/".format(dates[-1])).where(sql_where).cache()
cdf1.createOrReplaceTempView("cumu")

print "compare sum data: ", dates[-1]

# df1 = spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code from unified_data_view  group by app_id , device_code, country_code EXCEPT ALL select app_id , device_code, country_code, free_app_download from cumu  ").show()
# spark.sql("select app_id, free_app_download, device_code, country_code from cumu EXCEPT ALL select app_id , device_code, country_code , sum(free_app_download) as free_app_download  from unified_data_view  group by app_id , device_code, country_code   ").show()

spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code from union_unified_data  group by app_id , device_code, country_code order by app_id , device_code, country_code").createOrReplaceTempView("df1")
spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code from cumu order by app_id , device_code, country_code").createOrReplaceTempView("df2")
spark.sql("select * from df1 except all select * from df2").show()
spark.sql("select * from df2 except all select * from df1").show()



In [0]:

qa_df=spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code from union_unified_data  group by app_id , device_code, country_code order by app_id , device_code, country_code").cache()
qa_df.createOrReplaceTempView("df1")
dev_df = spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code from cumu order by app_id , device_code, country_code").cache()
dev_df.createOrReplaceTempView("df2")

spark.sql("select * from df1 where app_id=365691077 and device_code='ios-phone' and  country_code='WW'").show()
spark.sql("select * from df2 where app_id=365691077 and device_code='ios-phone' and  country_code='WW'").show()


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/2010
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2010-07-31/

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2010-07-04/device_code=ios-tablet/part-00000-bd81af73-bf68-47b1-a42a-df2df32c68df.c000.gz.parquet").printSchema()
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2010-07-04/device_code=ios-tablet").show()


In [0]:

import datetime

start = "2010-07-04"
end = "2013-08-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


test_ios_app_list = [377194688 , 375875657,366236510, 354990881,365691077 , 343200656 ,20600000009072,20600000000553]
device_code=['ios-phone','ios-tablet','ios-all','android-all']
country_list=['WW', 'US']
category_list=[100000, 100026,400025,400000]


# test_list = [(app, device, country ) for app in test_ios_app_list  for device in device_code for country in country_list ]
date_string=[d.strftime("%Y-%m-%d") for d in dates]
# print date_string

sql_where = "date between '{}' and '{}' and app_id in ({}) and device_code in ('{}') and country_code in ('{}') and category_id in ({}) ".format(start, dates[-1], ",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list), ",".join(map(str,category_list)))
print sql_where

result=list()
# for day in dates:
#     _unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={%s}" % (day.strftime("%Y-%m-%d"))).where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue", "category_id").collect()
#     # print _unified_data
#     result.extend(_unified_data)

_unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/").where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue","category_id")


# .format(",".join(date_string))).
# _unified_data = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/").where("granularity ='daily' and date between '{}' and '{}' and device_code in ('{}')".format(start,dates[-1],"','".join(device_code))).where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue","category_id")


# _unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=20{10,11,12,13}-*").where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue","category_id")


_unified_data.createOrReplaceTempView("unified_data_view")

# spark.createDataFrame(result).createOrReplaceTempView("unified_data_view")
# spark.sql("select app_id, sum(free_app_download) from unified_data_view group by app_id, device_code, country_code,category_id").show(2)

sql_where_cum = "app_id in ({}) and device_code in ('{}') and country_code in ('{}') and category_id in ({}) ".format( ",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list), ",".join(map(str,category_list)))

cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-cum.v1/fact/date={}/".format(dates[-1])).where(sql_where_cum).cache()
cdf1.createOrReplaceTempView("cumu")

print "compare sum data: ", dates[-1]

# df1 = spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code,category_id from unified_data_view group by app_id , device_code, country_code,category_id EXCEPT ALL select app_id , device_code, country_code, free_app_download , category_id from cumu  ").show()
# spark.sql("select app_id, free_app_download, device_code, country_code,category_id from cumu EXCEPT ALL select app_id , device_code, country_code , sum(free_app_download) as free_app_download ,category_id from unified_data_view  group by app_id , device_code, country_code,category_id   ").show()

spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code ,category_id from unified_data_view  group by app_id , device_code, country_code,category_id order by app_id , device_code, country_code, category_id").createOrReplaceTempView("df1")
spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code ,category_id from cumu order by app_id , device_code, country_code, category_id").createOrReplaceTempView("df2")
spark.sql("select * from df1 except all select * from df2").show()
spark.sql("select * from df2 except all select * from df1").show()



In [0]:

_unified_data.show()

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
import datetime

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()



start = "2010-07-04"
end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

est_list = [ "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates ]
rank_list = ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates  ]
# print est_list
# print rank_list






def test_count(test_date):
    # test_date = '2017-08-01'
    namespace = "aa.store.market-size.v1"
    
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": rank_list,
    
            }
            
        ]
    }
    sql_where = "app_id in ({}) and device_code in ('{}') and country_code in ('{}') and category_id in ({})".format(",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list), ",".join(map(str,category_list)))
    print sql_where

    
    sql_text = """
    
    
    -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download  , 
                                    store_unified.paid_app_download , 
                                    store_unified.revenue , 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download AS rank_free_app_download, 
                                    rank_unified.paid_app_download AS rank_paid_app_download, 
                                    rank_unified.revenue AS rank_revenue, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
                    WHERE %s
    
    
    
    """%(sql_where)
    


    run(spark, ingest_msg, sql_text)
 
    category_1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-cum.v1/fact").where("date='{}'".format(test_date)).cache()


    test_ios_app_list = [377194688 , 375875657,366236510, 354990881,365691077 , 343200656 ,20600000009072,20600000000553]
    device_code=['ios-phone','ios-tablet','ios-all','android-all']
    country_list=['WW', 'US']
    category_list=[100000, 100026,400025,400000]


    


    cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-cum.v1/fact/date={}/".format(dates[-1])).where(sql_where).cache()
    cdf1.createOrReplaceTempView("cumu")

    print "compare sum data: ", dates[-1]


    spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code ,category_id from unified_data_test where {}  group by app_id , device_code, country_code,category_id order by app_id , device_code, country_code, category_id".format(sql_where)).createOrReplaceTempView("df1")
    spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code ,category_id from cumu order by app_id , device_code, country_code, category_id").createOrReplaceTempView("df2")
    spark.sql("select * from df1 except all select * from df2").show()
    spark.sql("select * from df2 except all select * from df1").show()

test_count(dates[-1])


In [0]:

import datetime

start = "2010-07-04"
end = "2014-01-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


test_ios_app_list = [377194688 , 375875657,366236510, 354990881,365691077 , 343200656 ,20600000009072,20600000000553]
device_code=['ios-phone','ios-tablet','ios-all','android-all']
country_list=['WW', 'US']
# test_list = [(app, device, country ) for app in test_ios_app_list  for device in device_code for country in country_list ]


sql_where = "app_id in ({}) and device_code in ('{}') and country_code in ('{}')".format(",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list) )
print sql_where

result=list()
for day in dates:
    _unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % (day.strftime("%Y-%m-%d"))).where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue").collect()
    # print _unified_data
    result.extend(_unified_data)


spark.createDataFrame(result).createOrReplaceTempView("unified_data_view")
spark.sql("select app_id, sum(free_app_download) from unified_data_view group by app_id, device_code, country_code").show(2)



cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}/".format(dates[-1])).where(sql_where).cache()
cdf1.createOrReplaceTempView("cumu")

print "compare sum data: ", dates[-1]

# df1 = spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code from unified_data_view  group by app_id , device_code, country_code EXCEPT ALL select app_id , device_code, country_code, free_app_download from cumu  ").show()
# spark.sql("select app_id, free_app_download, device_code, country_code from cumu EXCEPT ALL select app_id , device_code, country_code , sum(free_app_download) as free_app_download  from unified_data_view  group by app_id , device_code, country_code   ").show()

spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code from unified_data_view  group by app_id , device_code, country_code order by app_id , device_code, country_code").createOrReplaceTempView("df1")
spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code from cumu order by app_id , device_code, country_code").createOrReplaceTempView("df2")
spark.sql("select * from df1 except all select * from df2").show()
spark.sql("select * from df2 except all select * from df1").show()


import datetime

start = "2010-07-04"
end = "2014-01-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


test_ios_app_list = [377194688 , 375875657,366236510, 354990881,365691077 , 343200656 ,20600000009072,20600000000553]
device_code=['ios-phone','ios-tablet','ios-all','android-all']
country_list=['WW', 'US']
# test_list = [(app, device, country ) for app in test_ios_app_list  for device in device_code for country in country_list ]


sql_where = "app_id in ({}) and device_code in ('{}') and country_code in ('{}')".format(",".join(map(str,test_ios_app_list)),   "','".join(device_code), "','".join(country_list) )
print sql_where

result=list()
for day in dates:
    _unified_data = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/").parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % (day.strftime("%Y-%m-%d"))).where(sql_where).select("app_id", "device_code", "country_code","free_app_download","paid_app_download","revenue").collect()
    # print _unified_data
    result.extend(_unified_data)


spark.createDataFrame(result).createOrReplaceTempView("unified_data_view")
spark.sql("select app_id, sum(free_app_download) from unified_data_view group by app_id, device_code, country_code").show(2)



cdf1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date={}/".format(dates[-1])).where(sql_where).cache()
cdf1.createOrReplaceTempView("cumu")

print "compare sum data: ", dates[-1]

# df1 = spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code from unified_data_view  group by app_id , device_code, country_code EXCEPT ALL select app_id , device_code, country_code, free_app_download from cumu  ").show()
# spark.sql("select app_id, free_app_download, device_code, country_code from cumu EXCEPT ALL select app_id , device_code, country_code , sum(free_app_download) as free_app_download  from unified_data_view  group by app_id , device_code, country_code   ").show()

spark.sql("select app_id, sum(free_app_download) as free_app_download, sum(paid_app_download) as paid_app_download, sum(revenue) as revenue, device_code, country_code from unified_data_view  group by app_id , device_code, country_code order by app_id , device_code, country_code").createOrReplaceTempView("df1")
spark.sql("select app_id, free_app_download, paid_app_download, revenue, device_code, country_code from cumu order by app_id , device_code, country_code").createOrReplaceTempView("df2")
spark.sql("select * from df1 except all select * from df2").show()
spark.sql("select * from df2 except all select * from df1").show()



In [0]:


# spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code from unified_data_view  group by app_id , device_code, country_code except all select app_id , device_code, country_code, free_app_download from cumu  ").show()
# spark.sql("select app_id, free_app_download, device_code, country_code from cumu except all select app_id , device_code, country_code , sum(free_app_download) as free_app_download  from unified_data_view  group by app_id , device_code, country_code   ").show()
# |354990881|            12027|             null|   null|  ios-phone|          WW|     100000|

a = spark.sql("select app_id, sum(free_app_download) as free_app_download, device_code, country_code , category_id from unified_data_view  group by app_id , device_code, country_code,category_id  order by app_id , category_id,device_code, country_code").createOrReplaceTempView("df1")
b = spark.sql("select app_id, free_app_download, device_code, country_code, category_id from cumu order by app_id , device_code, country_code, category_id ").createOrReplaceTempView("df2")
spark.sql("select * from df1 order by app_id , device_code, country_code, category_id").show()
spark.sql("select * from df2 order by app_id , device_code, country_code, category_id").show()

# |377194688|          2431365|  ios-phone|          US|
# |377194688|          2431365|  ios-phone|          US|
# |377194688|          2431365|    ios-all|          US|

